# Задача регрессии на реальных данных
Рассмотрим датасет с ценами на жильё в Москве за 2011-2015 год, данные предоставлены [Сбербанком](https://www.kaggle.com/c/sberbank-russian-housing-market/data), спасибо им за это.

А ещё:
- научимся работать с категориальными (номинальными) признаками
- поймём, почему важно нормализовать данные до того, как отправлять их считаться в модель
- посмотрим на альтернативы sklearn'овской LinearRegression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

На том же сайте есть [пример](https://www.kaggle.com/captcalculator/a-very-extensive-sberbank-exploratory-analysis) хорошего первичного анализа. Откроем таблицу:

In [ ]:
data = pd.read_csv('sberbank_moscow.csv')
data.head()

Как видим, признаков достаточно много (292), так что излюбленный метод `.info()` особо не поможет:

In [ ]:
data.info()

Посмотрим на количество пропусков по столбцам:

In [ ]:
data.isnull().sum()

Как видим, про некоторые признаки данных не очень много, так что выберем колонки, смысл которых мы заведомо будем понимать. 

In [ ]:
target_column = 'price_doc'

In [ ]:
nominal_feature_columns = [
    'sub_area',
    'ecology',
]

In [ ]:
data[nominal_feature_columns].isnull().sum()

In [ ]:
internal_feature_columns = [
    'full_sq',
    'floor',
]

In [ ]:
data[internal_feature_columns].isnull().sum()

In [ ]:
school_feature_columns = [
    'children_preschool',
    'children_school',
    'school_education_centers_top_20_raion',
    'university_top_20_raion',
    'additional_education_km',
    'university_km',
]

In [ ]:
data[school_feature_columns].isnull().sum()

In [ ]:
infrastructure_feature_columns = [
    'nuclear_reactor_km',
    'power_transmission_line_km',
    'public_transport_station_km',
    'public_transport_station_min_walk',
    'mkad_km',
    'kremlin_km',
]

In [ ]:
data[infrastructure_feature_columns].isnull().sum()

Посмотрели на колонки и пропуски в них, теперь объединим всё в один список и будем использовать в качестве признаков только перечисленные столбцы:

In [ ]:
feature_columns = nominal_feature_columns + internal_feature_columns + school_feature_columns + infrastructure_feature_columns + [target_column]
len(feature_columns)

Сохраните таблицу только с перечисленными колонками и удалите из неё все строки с пропусками:

In [ ]:
# YOUR CODE

## Обработка мультиколлинеарности
Посчитайте коэффициенты корреляции между колонками таблицы:

In [ ]:
numeric_feature_columns = internal_feature_columns + school_feature_columns + infrastructure_feature_columns + [target_column]
numeric_feature_columns

In [ ]:
# YOUR CODE

Если корреляция между признаками по модулю высокая, они будут "мешаться" друг другу при обучении линейной модели, потому что вносят похожую информацию. Из набора нескольких коррелирующих признаков следует оставлять один. Оставьте среди числовых признаков только те, которые слабо коррелируют между собой:

In [ ]:
# YOUR CODE

Помимо мультиколлинеарности следует смотреть на корреляцию признаков с целевой переменной. Посмотрите на корреляцию столбца `price_doc` со всеми остальными и оставьте среди них только те, которые оказывают достаточно сильное влияние:

In [ ]:
# YOUR CODE

Страшный, но, возможно, удобный способ вывести цветную корреляционную таблицу прямо в `pandas` (на семинаре по визуализации расскажем, как сделать проще, не бойтесь).

Содержательная часть только тут `data.corr()`, в остальное можно не вникать, это для красивого отображения.

In [ ]:
data.corr().style.format("{:.2}").background_gradient(cmap='coolwarm', axis=1)

## OHE aka One Hot Encoding
Что сейчас мешает просто взять и запустить LinearRegression? Дело в том, что три колонки содержат не понятные компьютеру числа, а текстовые значения, а именно:

In [ ]:
data[nominal_feature_columns]

Есть простой способ закодировать данные. Например, рассмотрим колонку `ecology`. Она принимает только следующие значения:

In [ ]:
np.unique(data.ecology)

Есть два варианта действий:
- просто заменить каждое значение на число (4 - 'excellent', 3 - 'good' etc.)
- рассмотреть каждое значение как отдельный признак и создать в таблице новые колонки с названиями этих значений

Подумайте, почему $1$й способ хорошо подойдёт для колонки `ecology`, а $2$й -- для других двух столбцов.

Создайте словарь, сопоставляющий числовые значения от $0$ до $4$ описанию экологии от `'no data'` до `'excellent'` соответственно:

In [ ]:
# YOUR CODE

Теперь можно заменить значения в колонке `ecology` на заданные нами выше с помощью метода `.replace`:

In [ ]:
data.ecology = data.ecology.replace(to_replace=ecology_dict)

In [ ]:
data.shape

In [ ]:
data

Для добавления новых колонок-признаков воспользуемся методом `.get_dummies`:

In [ ]:
data = pd.get_dummies(data, columns=['sub_area'])

In [ ]:
data.shape

In [ ]:
data.head()

## Обучение модели
Обучите модель линейной регрессии на полученных данных, посмотрите на полученное качество и коэффициенты. Для объективной оценки не забудьте разделить выборку на обучающую и контрольную.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
Y = data[target_column].values
X = data.drop(target_column, axis=1)
X.shape, Y.shape

In [ ]:
# Разбиваем данные на обучающую и тестовую часть.
# YOUR CODE

# Создание модели, которая будет подбирать веса для признаков.
# YOUR CODE

# Просим модель подобрать веса для признаков.
# YOUR CODE

# Предсказываем значения с помощью модели.
# На трейне.
# YOUR CODE
# На тесте.
# YOUR CODE

## Нормализация признаков
Казалось бы, чем больше коэффициент модели, тем сильнее признак влияет на предсказание.

Но давайте посмотрим на разброс признаков: некоторые принимают значения порядка $10-100$, а некоторые --- около $0$:

In [ ]:
data[numeric_feature_columns].describe()

И тогда понятно, почему у некоторых признаков коэффициент может оказаться сильно меньше, чем у других. Потому что модели приходится с помощью коэффициентов приводить значения к одной шкале.

Хорошим тоном является нормализация всех признаков. Например, можно привести значения каждого столбца к шкале $[0..1]$ с помощью следующего преобразования:
$$
\frac{x - min(x)}{max(x)}
$$

Сделать это можно как ручками, так и с помощью питона:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Создаём инструмент для нормализации признаков.
min_max_scaler = MinMaxScaler()
# Преобразуем признаки (на выходе будет np.ndarray).
X_scaled = min_max_scaler.fit_transform(X)
# Преобразуем np.ndarray обратно в pandas таблицу для удобства.
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

X_scaled.head()

In [ ]:
X_scaled.shape, Y.shape

---

---

Снова обучите модель, уже на новых данных после нормализации (не забудьте разбить на train и test нормализованную таблицу `X_scaled`!). Сравните качество и коэффициенты с тем, что получалось раньше.

Какой признак влияет на предсказание сильнее всего?

In [ ]:
# YOUR CODE

## Регрессия из библиотеки scipy
Конечно, линейная регрессия реализована не в одной питонячьей библиотеке. Например, есть такая версия из библиотеки `scipy`. Её достаточно сложно заставить предсказывать значения на новых (тестовых) данных, зато она предоставляет много статистической информации о процессе обучения и полученных коэффициентах модели.

In [ ]:
import statsmodels.api as sm


ols_model = sm.OLS(Y, X_scaled, hasconst=False)
ols_results = ols_model.fit()
print(ols_results.summary())

- Значение в `P>|t|` позволяет оценить **не**важность признака для модели (если значение близко к нулю, признак вносит значительный вклад в предсказания).
- `R-squared` (доля объяснённой дисперсии) позволяет оценить, насколько хорошо модель описывает данные (чем ближе к 1, тем лучше).
- `Prob (F-statistic)` говорит нам, насколько модель хуже, чем если бы мы просто положили все веса равными нулю (т.е. чем ближе это значение к 0, тем лучше наша модель по сравнению с константой).

Если не очень боитесь математическую статистику и теорвер, можете сходить за подробностями [сюда](http://efavdb.com/interpret-linear-regression/).